# IDAO: expected time of orders in airports

Airports are special points for taxi service. Every day a lot of people use a taxi to get to the city centre from the airport.

One of important task is to predict how long a driver need to wait an order. It helps to understand what to do. Maybe the driver have to wait near doors, or can drink a tea, or even should drive to city center without an order.

We request you to solve a simple version of this prediction task.

**Task:** predict time of $k$ orders in airport (time since now when you get an order if you are $k$-th in queue), $k$ is one of 5 values (different for every airports).

**Data**
- train: number of order for every minutes for 6 months
- test: every test sample has datetime info + numer of order for every minutes for last 2 weeks

**Submission:** for every airport you should prepare a model which will be evaluated in submission system (code + model files). You can make different models for different airports.

**Evaluation:** for every airport for every $k$ sMAPE will be calculated and averaged. General leaderboard will be calculated via Borda count. 

## Baseline

In [1]:
%pylab inline

import catboost
import pandas as pd
import pickle
import tqdm

Populating the interactive namespace from numpy and matplotlib


Let's prepare a model for set2.

# Load train dataset

In [2]:
# Нужно редактировать под каждую из задач: A, B и C

set_name = 'set1'
path_train_set = '../../data/train/{}.csv'.format(set_name)

data = pd.read_csv(path_train_set)
data.datetime = data.datetime.apply(
    lambda x: datetime.datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
data = data.sort_values('datetime')
data.head()

,datetime,num_orders
0,2018-03-01 00:00:00,0
1,2018-03-01 00:01:00,0
2,2018-03-01 00:02:00,0
3,2018-03-01 00:03:00,0
4,2018-03-01 00:04:00,1


Predict position for set2.

In [3]:
target_positions = {
    'set1': [10, 30, 45, 60, 75],
    'set2': [5, 10, 15, 20, 25],
    'set3': [5, 7, 9, 11, 13]
}[set_name]

Some useful constant.

In [4]:
HOUR_IN_MINUTES = 60
DAY_IN_MINUTES = 24 * HOUR_IN_MINUTES
WEEK_IN_MINUTES = 7 * DAY_IN_MINUTES

MAX_TIME = DAY_IN_MINUTES

## Generate train samples with targets

We have only history of orders (count of orders in every minutes) but we need to predict time of k orders since current minutes. So we should calculate target for train set. Also we will make a lot of samples from all set (we can only use two weeks of history while prediction so we can use only two weeks in every train sample).

In [5]:
samples = {
    'datetime': [],
    'history': []}

for position in target_positions:
    samples['target_{}'.format(position)] = []
    
num_orders = data.num_orders.values

To calculate target (minutes before k orders) we are going to use cumulative sum of orders. 

In [6]:
# start after 2 weeks because of history
# finish earlier because of target calculation
for i in range(2 * WEEK_IN_MINUTES,
               len(num_orders) - 2 * DAY_IN_MINUTES):
    
    samples['datetime'].append(data.datetime[i])
    samples['history'].append(num_orders[i-2*WEEK_IN_MINUTES:i])
    
    # cumsum not for all array because of time economy
    cumsum_num_orders = num_orders[i+1:i+1+2*DAY_IN_MINUTES].cumsum()
    for position in target_positions:
        orders_by_positions = np.where(cumsum_num_orders >= position)[0]
        if len(orders_by_positions):
            time = orders_by_positions[0] + 1
        else:
            # if no orders in last days
            time = MAX_TIME
        samples['target_{}'.format(position)].append(time)

Convert to pandas.dataframe. Now we have targets to train and predict.

In [7]:
df = pd.DataFrame.from_dict(samples)
df.head()

,datetime,history,target_10,target_30,target_45,target_60,target_75
0,2018-03-15 00:00:00,"[0, 0, 0, 0, 1, 2, 0, 1, 1, 4, 0, 1, 1, 1, 1, ...",5,18,28,32,42
1,2018-03-15 00:01:00,"[0, 0, 0, 1, 2, 0, 1, 1, 4, 0, 1, 1, 1, 1, 3, ...",5,19,27,32,42
2,2018-03-15 00:02:00,"[0, 0, 1, 2, 0, 1, 1, 4, 0, 1, 1, 1, 1, 3, 2, ...",7,20,27,33,43
3,2018-03-15 00:03:00,"[0, 1, 2, 0, 1, 1, 4, 0, 1, 1, 1, 1, 3, 2, 3, ...",7,21,26,35,42
4,2018-03-15 00:04:00,"[1, 2, 0, 1, 1, 4, 0, 1, 1, 1, 1, 3, 2, 3, 1, ...",7,20,26,35,42


# Train model

Let's generate simple features.

By time:

In [8]:
df['weekday'] = df.datetime.apply(lambda x: x.weekday())
df['hour'] = df.datetime.apply(lambda x: x.hour)
df['minute'] = df.datetime.apply(lambda x: x.minute)

Aggregators by order history with different shift and window size:

In [9]:
SHIFTS = [
    HOUR_IN_MINUTES // 4,
    HOUR_IN_MINUTES // 2,
    HOUR_IN_MINUTES,
    DAY_IN_MINUTES,
    DAY_IN_MINUTES * 2,
    WEEK_IN_MINUTES,
    WEEK_IN_MINUTES * 2
]

WINDOWS = [
    HOUR_IN_MINUTES // 4,
    HOUR_IN_MINUTES // 2,
    HOUR_IN_MINUTES,
    DAY_IN_MINUTES,
    DAY_IN_MINUTES * 2,
    WEEK_IN_MINUTES,
    WEEK_IN_MINUTES * 2
]

In [10]:
for shift in SHIFTS:
    for window in WINDOWS:
        if window >= shift:
            continue
        df['num_orders_{}_{}'.format(shift, window)] = \
            df.history.apply(lambda x: x[-shift : -shift + window].sum())

In [11]:
# ухудшает модель

"""
WINDOWS2 = np.ceil(df.loc[:, df.columns.str.startswith("target_")].mean(axis=0)).astype(int).tolist()

for shift in SHIFTS:
    for window in WINDOWS2:
        if window >= shift:
            continue
        df['num_orders_{}_{}'.format(shift, window)] = \
            df.history.apply(lambda x: x[-shift : -shift + window].sum())

WINDOWS2
"""

'\nWINDOWS2 = np.ceil(df.loc[:, df.columns.str.startswith("target_")].mean(axis=0)).astype(int).tolist()\n\nfor shift in SHIFTS:\n    for window in WINDOWS2:\n        if window >= shift:\n            continue\n        df[\'num_orders_{}_{}\'.format(shift, window)] =             df.history.apply(lambda x: x[-shift : -shift + window].sum())\n\nWINDOWS2\n'

In [12]:
# df.drop(columns=['num_orders_{}_{}'.format(shift, window)
#                  for shift in SHIFTS for window in WINDOWS2 if window < shift],
#         inplace=True)

In [13]:
def compute_target(history, prefix, count_first=True):
    values = {}
    
    cumsum_num_orders = history[:2 * DAY_IN_MINUTES].cumsum()
    
    variants = target_positions
    if count_first:
        variants = [1] + variants
    
    for position in variants:
        orders_by_positions = np.where(cumsum_num_orders >= position)[0]
        if len(orders_by_positions):
            time = orders_by_positions[0] + 1
        else:
            time = MAX_TIME
        values['{}_{}'.format(prefix, position)] = time

    return pd.Series(values)

In [14]:
def extra_features_history_1(history):
    return compute_target(history[::-1], prefix="order_last", count_first=True)

df = df.merge(df.history.apply(extra_features_history_1), left_index=True, right_index=True)

In [15]:
def extra_features_history_2(history):
    return compute_target(history[WEEK_IN_MINUTES:], prefix="order_middle_1week", count_first=False)

df = df.merge(df.history.apply(extra_features_history_2), left_index=True, right_index=True)

In [16]:
def extra_features_history_3(history):
    return compute_target(history, prefix="order_middle_2week", count_first=False)

df = df.merge(df.history.apply(extra_features_history_3), left_index=True, right_index=True)

In [17]:
def extra_features_history_4(history):
    return compute_target(history[-DAY_IN_MINUTES:], prefix="order_middle_1day", count_first=False)

df = df.merge(df.history.apply(extra_features_history_4), left_index=True, right_index=True)

In [18]:
# ухудшает модель

def extra_features_history_5(history):
    return compute_target(history[-2 * DAY_IN_MINUTES:], prefix="order_middle_2day", count_first=False)

# df = df.merge(df.history.apply(extra_features_history_5), left_index=True, right_index=True)

In [19]:
# df.drop(columns=df.columns[df.columns.str.startswith('order_middle_2day_')], inplace=True)

In [20]:
# дают небольшое улучшение, которое можно списать на шум

def extra_features_history_6(history):
    return compute_target(history[-3 * DAY_IN_MINUTES:], prefix="order_middle_3day", count_first=False)

df = df.merge(df.history.apply(extra_features_history_6), left_index=True, right_index=True)

In [21]:
# df.drop(columns=df.columns[df.columns.str.startswith('order_middle_3day_')], inplace=True)

In [22]:
def max_empty_window(history, prefix):
    res = np.nonzero(history)[0]
    res = res[1:] - res[:-1]
    return pd.Series({
        "{}_min".format(prefix) : res.min() if len(res) else np.nan,
        "{}_max".format(prefix) : res.max() if len(res) else np.nan,
        "{}_med".format(prefix) : np.median(res) if len(res) else np.nan,
        "{}_num".format(prefix) : res.shape[0],
    })

In [23]:
# дают небольшое улучшение, которое можно списать на шум

def max_empty_stats_1(history):
    return max_empty_window(history[-HOUR_IN_MINUTES:], "max_empty_stats_1hour")

df = df.merge(df.history.apply(max_empty_stats_1), left_index=True, right_index=True)

In [24]:
# дают небольшое улучшение, которое можно списать на шум

def max_empty_stats_2(history):
    return max_empty_window(history[-2 * HOUR_IN_MINUTES:], "max_empty_stats_2hour")

df = df.merge(df.history.apply(max_empty_stats_2), left_index=True, right_index=True)

In [25]:
# дают небольшое улучшение, которое можно списать на шум

def max_empty_stats_3(history):
    return max_empty_window(history[-HOUR_IN_MINUTES // 2:], "max_empty_stats_30min")

df = df.merge(df.history.apply(max_empty_stats_3), left_index=True, right_index=True)

In [26]:
# дают небольшое улучшение, которое можно списать на шум

def max_empty_stats_4(history):
    return max_empty_window(history[-WEEK_IN_MINUTES-HOUR_IN_MINUTES:
                                    -WEEK_IN_MINUTES+HOUR_IN_MINUTES], "max_empty_stats_middle_2hour")

df = df.merge(df.history.apply(max_empty_stats_4), left_index=True, right_index=True)

In [27]:
# df.drop(columns=df.columns[df.columns.str.startswith('max_empty_stats_')], inplace=True)

In [28]:
holidays = {
    '01-01': {'holiday', 'dayoff'},
    '01-02': {'dayoff'},
    '01-03': {'dayoff'},
    '01-04': {'dayoff'},
    '01-05': {'dayoff'},
    '01-06': {'dayoff'},
    '01-07': {'holiday', 'dayoff'},
    '02-23': {'holiday', 'dayoff'},
    '02-24': {'dayoff'},
    '03-08': {'holiday', 'dayoff'},
    '05-01': {'holiday', 'dayoff'},
    '05-08': {'dayoff'},
    '05-09': {'holiday', 'dayoff'},
    '11-06': {'dayoff'},
    '12-31': {'holiday', 'dayoff'}
}

holidays_good_arrive = {
    '01-01': {'holiday', 'dayoff'},
    '01-02': {'dayoff'},
    '02-22': {'holiday', 'dayoff'},
    '02-23': {'holiday', 'dayoff'},
    '03-07': {'holiday', 'dayoff'},
    '03-08': {'holiday', 'dayoff'},
    '05-01': {'holiday', 'dayoff'},
    '04-31': {'holiday', 'dayoff'},
    '05-02': {'holiday', 'dayoff'},
    '05-08': {'dayoff'},
    '05-09': {'holiday', 'dayoff'},
    '11-06': {'dayoff'},
    '12-31': {'holiday', 'dayoff'}
}

In [29]:
df['holiday'] = df.datetime.apply(lambda x: '{:%m-%d}'.format(x) in holidays)
df['good_arrive'] = df.datetime.apply(lambda x: '{:%m-%d}'.format(x) in holidays_good_arrive)

In [30]:
def count_std_sum(history, shift, duration):
    return np.std(np.sum(history.reshape(-1, shift)[:, -duration:], axis = 1))

for position in target_positions:
    df['sum_std_day_' + str(position)] = \
        df.apply(lambda r: count_std_sum(r['history'], 60 * 24,
                                         r['order_last_'+str(position)]),
                 axis=1)

In [31]:
df.columns

Index(['datetime', 'history', 'target_10', 'target_30', 'target_45',
       'target_60', 'target_75', 'weekday', 'hour', 'minute',
       'num_orders_30_15', 'num_orders_60_15', 'num_orders_60_30',
       'num_orders_1440_15', 'num_orders_1440_30', 'num_orders_1440_60',
       'num_orders_2880_15', 'num_orders_2880_30', 'num_orders_2880_60',
       'num_orders_2880_1440', 'num_orders_10080_15', 'num_orders_10080_30',
       'num_orders_10080_60', 'num_orders_10080_1440', 'num_orders_10080_2880',
       'num_orders_20160_15', 'num_orders_20160_30', 'num_orders_20160_60',
       'num_orders_20160_1440', 'num_orders_20160_2880',
       'num_orders_20160_10080', 'order_last_1', 'order_last_10',
       'order_last_30', 'order_last_45', 'order_last_60', 'order_last_75',
       'order_middle_1week_10', 'order_middle_1week_30',
       'order_middle_1week_45', 'order_middle_1week_60',
       'order_middle_1week_75', 'order_middle_2week_10',
       'order_middle_2week_30', 'order_middle_2week_45

In [32]:
df.drop(columns=['datetime', 'history']).to_hdf('prepared_data/train_{}.hdf'.format(set_name), 'key')

Train/validation split for time. Let's use last 4 weeks for validation.

In [33]:
df.datetime.min(), df.datetime.max()

(Timestamp('2018-03-15 00:00:00'), Timestamp('2018-08-29 23:59:00'))

In [34]:
df_train = df.loc[df.datetime <= df.datetime.max() - datetime.timedelta(days=28)]
df_valid = df.loc[df.datetime  > df.datetime.max() - datetime.timedelta(days=28)]

In [35]:
df_train = df
df_valid = None

In [36]:
from functools import reduce
import operator

mask = reduce(operator.or_, [
    df.columns.str.startswith('target_'),
    [c in {'datetime', 'history'} for c in df.columns],
])

features = df.columns[~mask].tolist()
features

['weekday',
 'hour',
 'minute',
 'num_orders_30_15',
 'num_orders_60_15',
 'num_orders_60_30',
 'num_orders_1440_15',
 'num_orders_1440_30',
 'num_orders_1440_60',
 'num_orders_2880_15',
 'num_orders_2880_30',
 'num_orders_2880_60',
 'num_orders_2880_1440',
 'num_orders_10080_15',
 'num_orders_10080_30',
 'num_orders_10080_60',
 'num_orders_10080_1440',
 'num_orders_10080_2880',
 'num_orders_20160_15',
 'num_orders_20160_30',
 'num_orders_20160_60',
 'num_orders_20160_1440',
 'num_orders_20160_2880',
 'num_orders_20160_10080',
 'order_last_1',
 'order_last_10',
 'order_last_30',
 'order_last_45',
 'order_last_60',
 'order_last_75',
 'order_middle_1week_10',
 'order_middle_1week_30',
 'order_middle_1week_45',
 'order_middle_1week_60',
 'order_middle_1week_75',
 'order_middle_2week_10',
 'order_middle_2week_30',
 'order_middle_2week_45',
 'order_middle_2week_60',
 'order_middle_2week_75',
 'order_middle_1day_10',
 'order_middle_1day_30',
 'order_middle_1day_45',
 'order_middle_1day_60',


In [37]:
len(features)

73

In [38]:
def sMAPE(y_true, y_pred, shift=0):
    return 2 * np.mean(np.abs(y_true - y_pred) / (np.abs(y_true) + np.abs(y_pred) + shift))

Also we will save models for prediction stage.

In [39]:
from collections import defaultdict

model_to_save = { 'models': defaultdict(list) }

What is good or bad model? We can compare our model with constant solution. For instance median (optimal solution for MAE).

In [40]:
from sklearn.model_selection import KFold

In [41]:
%%time

score_pos = []

kf = KFold(n_splits=7, shuffle=True, random_state=4986)

np.random.seed(5872)
seeds = np.random.randint(1_000, 10_000, size=7)
print("seeds:", *seeds, end='\n\n')

cat_features = [
    df_train.columns.tolist().index('weekday'),
    df_train.columns.tolist().index('hour')
]

for position in target_positions:
    position_t = 'target_{}'.format(position)
    print(position_t)
    
    scores = []
    
    steps = 0
    
    for train_i, valid_i in kf.split(df_train):        
        X_train, y_train = df_train.loc[train_i, features], df_train.loc[train_i, position_t]
        X_valid, y_valid = df_train.loc[valid_i, features], df_train.loc[valid_i, position_t]
        
        model = catboost.CatBoostRegressor(
            iterations=2000, learning_rate=1.0, random_state=seeds[steps],
            loss_function='MAE', task_type='GPU', devices='3'
        )
        
        model.fit(X_train, y_train, cat_features=cat_features,
                  use_best_model=True, eval_set=(X_valid, y_valid), verbose=False)
        
        if df_valid is not None:
            X_valid, y_valid = df_valid.loc[:, features], df_valid.loc[:, position_t]
            
        y_pred = model.predict(X_valid)
        score = sMAPE(y_valid, y_pred)
        print('sMAPE = {:.6f}'.format(score))
        scores.append(score)
    
        model_to_save['models'][position].append(model)
        steps += 1
    
    score = np.mean(scores)
    print('MEAN:', score, '\n')
    score_pos.append(score)
    
score_pos

seeds: 1925 7354 7914 4109 2190 5034 4521

target_10
sMAPE = 0.256709
sMAPE = 0.260811
sMAPE = 0.261517
sMAPE = 0.260346
sMAPE = 0.259740
sMAPE = 0.257021
sMAPE = 0.258760
MEAN: 0.25927197475176866 

target_30
sMAPE = 0.180635
sMAPE = 0.182041
sMAPE = 0.179706
sMAPE = 0.181051
sMAPE = 0.180349
sMAPE = 0.183305
sMAPE = 0.181989
MEAN: 0.18129650870761618 

target_45
sMAPE = 0.157164
sMAPE = 0.159177
sMAPE = 0.156611
sMAPE = 0.157615
sMAPE = 0.157101
sMAPE = 0.157755
sMAPE = 0.157454
MEAN: 0.1575538466018225 

target_60
sMAPE = 0.141071
sMAPE = 0.141734
sMAPE = 0.139956
sMAPE = 0.140946
sMAPE = 0.140000
sMAPE = 0.141290
sMAPE = 0.141297
MEAN: 0.14089907624568915 

target_75
sMAPE = 0.128621
sMAPE = 0.129374
sMAPE = 0.127476
sMAPE = 0.128458
sMAPE = 0.128107
sMAPE = 0.127447
sMAPE = 0.129331
MEAN: 0.12840201419379618 

CPU times: user 53min 54s, sys: 16min 49s, total: 1h 10min 44s
Wall time: 24min 12s


In [42]:
np.mean(score_pos)

0.17348468410013854

Our model is better than constant solution. Saving model.

In [43]:
pickle.dump(model_to_save, open('models-{}.pkl'.format(set_name), 'wb'))